# Pengujian Model Apple Quality dengan TensorFlow Serving

Notebook ini digunakan untuk menguji model klasifikasi kualitas apel yang telah di-deploy menggunakan TensorFlow Serving. Model ini dapat memprediksi apakah kualitas apel "good" atau "bad" berdasarkan fitur-fitur seperti ukuran, berat, tingkat kemanisan, kerenyahan, kejusan, kematangan, dan keasaman.

## Alur Pengujian

1. **Persiapan Model**: 
   - Model di-deploy menggunakan Docker dan TensorFlow Serving
   - Akses melalui REST API di port 8501

2. **Preprocessing Data**:
   - Mengubah data mentah menjadi format yang sesuai dengan input model
   - Menggunakan transformasi yang sama dengan yang digunakan saat training

3. **Prediksi**:
   - Mengirim data ke TensorFlow Serving melalui REST API
   - Menerima hasil prediksi dalam format JSON

4. **Interpretasi Hasil**:
   - Mengubah nilai prediksi menjadi label "good" atau "bad"
   - Menghitung probabilitas hasil prediksi

## Cara Menjalankan Testing

1. **Build dan Jalankan Docker Container**:
   ```bash
   docker build -t apple-quality-serving .
   docker run -p 8501:8501 -p 8500:8500 apple-quality-serving
   ```

2. **Jalankan Notebook Testing**:
   - Jalankan sel kode di bawah untuk melakukan prediksi
   - Lihat hasil dan interpretasinya

## Catatan Penting

- Pastikan Docker dan TensorFlow Serving berjalan dengan benar
- Struktur direktori model harus sesuai dengan yang diharapkan oleh TensorFlow Serving
- Preprocessing data harus konsisten dengan yang digunakan saat training


In [10]:
import json
import requests
import numpy as np
import pandas as pd
from apple_transform import NUMERIC_FEATURES, transformed_name

# URL untuk REST API TensorFlow Serving
SERVER_URL = 'http://localhost:8501/v1/models/apple_quality_model:predict'

# Fungsi untuk melakukan preprocessing data
def preprocess_data(data):
    # Normalisasi data menggunakan mean dan std yang sama dengan yang digunakan saat training
    # Catatan: Dalam produksi sebenarnya, nilai ini harus diambil dari transform_fn
    
    # Contoh data yang sudah dinormalisasi (simulasi)
    processed_data = {}
    
    # Proses fitur numerik
    for feature in NUMERIC_FEATURES:
        processed_data[transformed_name(feature)] = [float(data[feature])]
    
    # Proses Acidity
    processed_data[transformed_name('Acidity')] = [float(data['Acidity'])]
    
    return processed_data

# Fungsi untuk melakukan prediksi
def predict_quality(data):
    print("=== PROSES PREDIKSI KUALITAS APEL ===")
    
    # Step 1: Preprocessing data
    print("\n1. Preprocessing Data:")
    print(f"Data mentah: {data}")
    processed_data = preprocess_data(data)
    print(f"Data setelah preprocessing: {processed_data}")
    
    # Step 2: Format data untuk TensorFlow Serving
    print("\n2. Memformat Data untuk TensorFlow Serving:")
    instances = {
        'instances': [processed_data]
    }
    print(f"Request JSON: {json.dumps(instances, indent=2)}")
    
    # Step 3: Kirim request ke server
    print("\n3. Mengirim Request ke TensorFlow Serving:")
    print(f"URL: {SERVER_URL}")
    response = requests.post(SERVER_URL, json=instances)
    print(f"Status Code: {response.status_code}")
    
    # Step 4: Parse response
    print("\n4. Menerima dan Parsing Response:")
    print(f"Response Raw: {response.text}")
    result = json.loads(response.text)
    print(f"Response JSON: {json.dumps(result, indent=2)}")
    prediction = result['predictions'][0][0]
    print(f"Nilai Prediksi: {prediction}")
    
    # Step 5: Konversi probability ke label
    print("\n5. Interpretasi Hasil:")
    label = "good" if prediction > 0.5 else "bad"
    probability = prediction if prediction > 0.5 else 1 - prediction
    print(f"Label: {label} (Probability: {probability:.4f})")
    
    return {
        "label": label,
        "probability": probability,
        "raw_prediction": prediction
    }

# Contoh penggunaan
sample_apple = {
    "A_id": 1,
    "Size": 0.6,
    "Weight": 0.6,
    "Sweetness": 0.7, 
    "Crunchiness": 0.8,
    "Juiciness": 0.9,
    "Ripeness": 0.7,
    "Acidity": 0.5
}

# Lakukan prediksi
try:
    print("\n=== MULAI PROSES TESTING ===")
    print("Menjalankan prediksi untuk sampel apel...")
    result = predict_quality(sample_apple)
    print("\n=== HASIL AKHIR ===")
    print(f"Prediction: {result['label']}")
    print(f"Probability: {result['probability']:.4f}")
    print(f"Raw prediction value: {result['raw_prediction']:.4f}")
    print("=== PROSES TESTING SELESAI ===")
except Exception as e:
    print(f"Error during prediction: {e}")
    print("Note: Make sure TensorFlow Serving is running with the correct model.")
    print("Run the following command to start TensorFlow Serving:")
    print("docker build -t apple-quality-serving . && docker run -p 8501:8501 -p 8500:8500 apple-quality-serving")


=== MULAI PROSES TESTING ===
Menjalankan prediksi untuk sampel apel...
=== PROSES PREDIKSI KUALITAS APEL ===

1. Preprocessing Data:
Data mentah: {'A_id': 1, 'Size': 0.6, 'Weight': 0.6, 'Sweetness': 0.7, 'Crunchiness': 0.8, 'Juiciness': 0.9, 'Ripeness': 0.7, 'Acidity': 0.5}
Data setelah preprocessing: {'A_id_xf': [1.0], 'Size_xf': [0.6], 'Weight_xf': [0.6], 'Sweetness_xf': [0.7], 'Crunchiness_xf': [0.8], 'Juiciness_xf': [0.9], 'Ripeness_xf': [0.7], 'Acidity_xf': [0.5]}

2. Memformat Data untuk TensorFlow Serving:
Request JSON: {
  "instances": [
    {
      "A_id_xf": [
        1.0
      ],
      "Size_xf": [
        0.6
      ],
      "Weight_xf": [
        0.6
      ],
      "Sweetness_xf": [
        0.7
      ],
      "Crunchiness_xf": [
        0.8
      ],
      "Juiciness_xf": [
        0.9
      ],
      "Ripeness_xf": [
        0.7
      ],
      "Acidity_xf": [
        0.5
      ]
    }
  ]
}

3. Mengirim Request ke TensorFlow Serving:
URL: http://localhost:8501/v1/models/app